# Exploratory Data Analysis

In this notebook some initial exploratory data analysis are performed. In particular, we are interested in the number of `null` values per column and the number of duplicated rows.

## Import required modules

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StringType, IntegerType, StructType, StructField
import pyspark.sql.functions as F
import dataio

## Create SparkSession

In [20]:
spark = (SparkSession.builder
            .master("local[*]")
            .appName("EDA")
            .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:2.7.0")
            .getOrCreate()
)

spark.sparkContext.setLogLevel("ERROR")

## Read US demographics data

In [6]:
dem = dataio.read_demographics_raw(spark, "./data/raw/demographics/us-cities-demographics.csv")

print(f"{dem.count():,} rows")
print(f"{len(dem.columns)} columns")
print()
print(dem.columns)

2,891 rows
12 columns

['City', 'State', 'Median Age', 'Male Population', 'Female Population', 'Total Population', 'Number of Veterans', 'Foreign-born', 'Average Household Size', 'State Code', 'Race', 'Count']


### Check number of `null` values per column

In [7]:
dem_null_counts = {col: dem.filter(dem[col].isNull()).count() for col in dem.columns}

dem_null_counts

{'City': 0,
 'State': 0,
 'Median Age': 0,
 'Male Population': 3,
 'Female Population': 3,
 'Total Population': 0,
 'Number of Veterans': 13,
 'Foreign-born': 13,
 'Average Household Size': 16,
 'State Code': 0,
 'Race': 0,
 'Count': 0}

It can be seen that the dataset is in pretty good shape. The column of interest (`City`, `State`, `Total population`, `Race`, `Count`) don't have any missing values, so we don't want to remove any rows.

We can however check some of the rows in which we have `null` entries.

In [10]:
dem_pd = (
    dem
    .where(
        F.col("Male Population").isNull() | 
        F.col("Female Population").isNull() | 
        F.col("Foreign-born").isNull()
    )
    .toPandas()
)

dem_pd.sample(10)

,City,State,Median Age,Male Population,Female Population,Total Population,Number of Veterans,Foreign-born,Average Household Size,State Code,Race,Count
12,Carolina,Puerto Rico,42.0,64758,77308,142066,None,None,None,PR,Hispanic or Latino,139967
11,Guaynabo,Puerto Rico,42.2,33066,37426,70492,None,None,None,PR,American Indian and Alaska Native,589
14,Guaynabo,Puerto Rico,42.2,33066,37426,70492,None,None,None,PR,Hispanic or Latino,69936
15,Ponce,Puerto Rico,40.5,56968,64615,121583,None,None,None,PR,Hispanic or Latino,120705
9,Caguas,Puerto Rico,40.4,34743,42265,77008,None,None,None,PR,American Indian and Alaska Native,624
5,San Juan,Puerto Rico,41.4,155408,186829,342237,None,None,None,PR,Hispanic or Latino,335559
7,The Villages,Florida,70.5,None,None,72590,15231,4034,None,FL,White,72211
10,The Villages,Florida,70.5,None,None,72590,15231,4034,None,FL,Hispanic or Latino,1066
13,Carolina,Puerto Rico,42.0,64758,77308,142066,None,None,None,PR,American Indian and Alaska Native,12143
0,Mayagüez,Puerto Rico,38.1,30799,35782,66581,None,None,None,PR,Asian,235


### Check duplicate rows

In [8]:
dem_dups = (dem
            .groupBy(dem.columns)
            .agg(F.count("*").cast("int").alias("n_dups"))
            .where(F.col("n_dups") > 1)
            .select(F.sum("n_dups"), F.count("n_dups"))
)

dem_dups.show()

+-----------+-------------+
|sum(n_dups)|count(n_dups)|
+-----------+-------------+
|       null|            0|
+-----------+-------------+



## Read US Immigration data

In [16]:
imm = dataio.read_immigration_raw(spark, "./data/raw/immigration/i94_data.parquet")

print(f"{imm.count():,} rows")
print(f"{len(imm.columns)} columns")
print()
print(imm.columns)

40,790,529 rows
28 columns

['cicid', 'i94yr', 'i94cit', 'i94res', 'i94port', 'arrdate', 'i94mode', 'i94addr', 'depdate', 'i94bir', 'i94visa', 'count', 'dtadfile', 'visapost', 'occup', 'entdepa', 'entdepd', 'entdepu', 'matflag', 'biryear', 'dtaddto', 'gender', 'insnum', 'airline', 'admnum', 'fltno', 'visatype', 'i94mon']


### Count `null` values per column

In [13]:
imm_null_counts = {c: imm.filter(F.col(c).isNull()).count() for c in imm.columns}

imm_null_counts

{'cicid': 0,
 'i94yr': 0,
 'i94cit': 28575,
 'i94res': 0,
 'i94port': 0,
 'arrdate': 0,
 'i94mode': 73949,
 'i94addr': 2027926,
 'depdate': 3308012,
 'i94bir': 9517,
 'i94visa': 0,
 'count': 0,
 'dtadfile': 131050,
 'visapost': 24032175,
 'occup': 40597574,
 'entdepa': 2404,
 'entdepd': 3287909,
 'entdepu': 40777323,
 'matflag': 3219581,
 'biryear': 9517,
 'dtaddto': 101551,
 'gender': 4079983,
 'insnum': 35710667,
 'airline': 1308066,
 'admnum': 0,
 'fltno': 333922,
 'visatype': 0,
 'i94mon': 0}

In this dataset we have many more `null` values. However, we are only interested in a few columns, so below we select only a subset of columns and drop all the rows that have any `null` entries. This is probably too restrictive, but given the amount of data available is okay.

In [17]:
keep_cols = [
    'cicid', 'i94yr', 'i94mon', 'i94cit', 'i94res', 'arrdate', 'i94addr', 'depdate', 
    'i94bir', 'biryear', 'gender', 'count', 'dtaddto', 'i94visa', 'visatype', 'admnum'
]

imm = imm.select(*keep_cols).dropna()

print(f"{imm.count():,} rows")
print(f"{len(imm.columns)} columns")

31,927,685 rows
16 columns


### Check for duplicate rows

Given the size of the data and that many columns have `null` entries we are going to check for duplicates only in a subset of the columns.

>__NOTE:__ Below cell will take a long time to run, depending on the amount of the memory available.

In [21]:
imm_dups = (imm
            .groupBy("cicid", "arrdate", "admnum")
            .agg(F.count("*").cast("int").alias("n_dups"))
            .where(F.col("n_dups") > 1)
            .select(F.sum("n_dups"), F.count("n_dups"))
)

imm_dups.show()

+-----------+-------------+
|sum(n_dups)|count(n_dups)|
+-----------+-------------+
|       null|            0|
+-----------+-------------+



Below we can show a few rows of the raw data. 

>__NOTE:__ At this stage the data types were already fixed when the `sas` data was written to `parquet`.

In [26]:
imm.limit(10).toPandas()

,cicid,i94yr,i94mon,i94cit,i94res,arrdate,i94addr,depdate,i94bir,biryear,gender,count,dtaddto,i94visa,visatype,admnum
0,4444623,2016,7,254,276,20654,CA,20657,23,1993,U,1,08312016,2,GMT,2147483647
1,4444624,2016,7,254,276,20654,GA,20657,27,1989,F,1,08312016,2,GMT,2147483647
2,4444625,2016,7,254,276,20654,MA,20657,52,1964,F,1,08312016,2,GMT,2147483647
3,4444626,2016,7,254,276,20654,NY,20656,44,1972,M,1,09012016,2,GMT,2147483647
4,4444627,2016,7,254,276,20654,FL,20658,43,1973,F,1,09012016,2,GMT,2147483647
5,4444629,2016,7,254,276,20654,CA,20658,7,2009,F,1,09012016,2,GMT,2147483647
6,4444630,2016,7,254,276,20654,FL,20658,15,2001,F,1,09012016,2,GMT,2147483647
7,4444631,2016,7,254,276,20654,NY,20658,17,1999,F,1,09012016,2,GMT,2147483647
8,4444632,2016,7,254,276,20654,GU,20657,34,1982,F,1,09012016,2,GMT,2147483647
9,4444633,2016,7,254,276,20654,GU,20657,44,1972,M,1,09012016,2,GMT,2147483647


## Data Processing Steps

At this stage we can proceed with the ETL steps.